In [1]:
print("ok")

ok


In [2]:
%pwd

'c:\\Users\\valib\\OneDrive\\Desktop\\My projects\\Medical-Chatbot\\research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\valib\\OneDrive\\Desktop\\My projects\\Medical-Chatbot'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [14]:
extracted_data=load_pdf_file(data='Data/')

In [16]:
#extracted_data

In [17]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [18]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 5860


In [20]:
#text_chunks

In [21]:
from langchain.embeddings import HuggingFaceEmbeddings

In [22]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [23]:
embeddings = download_hugging_face_embeddings()

C:\Users\valib\AppData\Local\Temp\ipykernel_15528\2661704553.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
c:\Users\valib\anaconda3\envs\medibot\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [24]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [26]:
#query_result

In [27]:
from dotenv import load_dotenv
load_dotenv()

True

In [28]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
GROQ_API_KEY=os.environ.get('GROQ_API_KEY')

In [29]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalbot-final"

pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

In [30]:
print(GROQ_API_KEY)  # This should print the API key, not 'None'


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


In [31]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

In [32]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [33]:
docsearch

In [34]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [35]:
retrieved_docs = retriever.invoke("What is Acne?")

In [36]:
retrieved_docs

[Document(id='834667b7-ff3c-4684-b9b3-7033797104a7', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='604cc982-6eb6-4e87-b114-86465a91fb38', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 38.0, 'page_label': '39', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Medical_book.pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed.(Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM -

In [38]:
import os

from groq import Groq

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explain the importance of fast language models",
        }
    ],
    model="llama-3.3-70b-versatile",
)

print(chat_completion.choices[0].message.content)

Fast language models are crucial in modern natural language processing (NLP) as they enable efficient and effective processing of human language. Here are some key reasons why fast language models are important:

1. **Real-time Applications**: Fast language models can handle real-time applications such as chatbots, virtual assistants, and voice-activated devices. They can quickly process and respond to user input, creating a seamless user experience.
2. **Large-Scale Text Analysis**: Fast language models can process vast amounts of text data quickly, making them ideal for large-scale text analysis tasks such as sentiment analysis, entity recognition, and topic modeling.
3. **Improved User Engagement**: Fast language models can enhance user engagement by providing quick and accurate responses to user queries. This is particularly important in applications such as customer service, where timely responses can significantly impact user satisfaction.
4. **Competitive Advantage**: Organizati

In [40]:
import os
from langchain_groq import ChatGroq

# Set the API key manually
os.environ["GROQ_API_KEY"] = "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"

# Initialize the Groq LLM
llm = ChatGroq(
    model_name="llama3-8b-8192",  # or "llama3-70b-8192"
    temperature=0.4,
    max_tokens=500,
    api_key=os.getenv("GROQ_API_KEY")  # Fetching API key
)

# Example Query
response = llm.invoke("What is the latest advancement in AI?")
print(response)


content='There have been many recent advancements in Artificial Intelligence (AI), and it\'s difficult to pinpoint a single "latest" one. However, here are some of the most significant developments in the field:\n\n1. **AlphaFold**: In 2020, Google\'s DeepMind AI system, AlphaFold, was able to predict the 3D structure of proteins with unprecedented accuracy. This breakthrough has the potential to revolutionize our understanding of biology and medicine.\n2. **Transformers**: The Transformer architecture, introduced in 2017, has become a standard component in many AI systems. It\'s particularly effective for natural language processing tasks, such as language translation, text summarization, and chatbots.\n3. **Generative Adversarial Networks (GANs)**: GANs have made significant progress in generating realistic images, videos, and music. They\'re being used in applications like data augmentation, image synthesis, and style transfer.\n4. **Explainable AI (XAI)**: As AI systems become more

In [41]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
import os

# Set Groq API Key (Ensure you have set it in environment variables)
os.environ["GROQ_API_KEY"] = "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"

# Define System Prompt
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

# Define Chat Prompt Template
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

# Initialize Groq LLM
llm = ChatGroq(
    model_name="llama3-8b-8192",  # or "llama3-70b-8192"
    temperature=0.4,
    max_tokens=500,
    api_key=os.getenv("GROQ_API_KEY")
)


In [42]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [43]:
response = rag_chain.invoke({"input": "what is Acromegaly and gigantism?"})
print(response["answer"])

Acromegaly is a disorder in which the abnormal release of a particular chemical from the pituitary gland in the brain causes increased growth in bone and soft tissue, as well as a variety of other disturbances throughout the body. Gigantism is a similar condition, but it occurs when the abnormality occurs before bone growth stops, resulting in unusual height.


In [44]:
response = rag_chain.invoke({"input": "What is stats?"})
print(response["answer"])

I don't know. The context provided doesn't mention "stats". It appears to be discussing a blood count test and athletic heart syndrome.
